<a href="https://colab.research.google.com/github/amaj8/Downton-Chatbot/blob/master/Downton_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from datetime import datetime as dt
import datetime
import re
from collections import Counter
import random
import itertools
import os

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

In [ ]:
!git clone https://github.com/amaj8/Downton-Abbey-Subtitles.git

Cloning into 'Downton-Abbey-Subtitles'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 69 (delta 0), reused 69 (delta 0), pack-reused 0
Unpacking objects: 100% (69/69), done.


* Preprocess the data
* Model
* Loss function

In [ ]:
def dialogue_pairs(group):
  pairs = []
  for i in range(1,len(group)):
    if dt.strptime(group[i][1].split(" --> ")[0],'%H:%M:%S,%f') <= dt.strptime(group[i-1][1].split(" --> ")[1] \
                        ,'%H:%M:%S,%f') + datetime.timedelta(seconds=4):
      pairs.append([group[i-1][2],group[i][2]])
  return pairs

In [ ]:
path = 'Downton-Abbey-Subtitles/subtitles'
pairs1 = []
count = 0
total = 0
troublesome_subs = []
for f in os.listdir(path):
  for sub in os.listdir(path+'/'+f):
    # print(sub)
    total += 1
    sub = open(path+'/'+f+'/'+sub,'r')
    try:
      content = sub.read()
      pairs1.extend(dialogue_pairs([group.split("\n",2) for group in content.split("\n\n")]))
    except:
      count += 1
      troublesome_subs.append(sub)
    
print(count/total * 100.)

3.3333333333333335


In [ ]:
print(count)

2


In [ ]:
print(troublesome_subs)

[<_io.TextIOWrapper name='Downton-Abbey-Subtitles/subtitles/Downton_Abbey - season 1.en/Downton Abbey - 1x07 - Episode 7.HDTV.FoV.en.srt' mode='r' encoding='UTF-8'>, <_io.TextIOWrapper name='Downton-Abbey-Subtitles/subtitles/Downton_Abbey - season 1.en/Downton Abbey - 1x03 - Episode 3.HDTV.RiVER.en.srt' mode='r' encoding='UTF-8'>]


In [ ]:

# f = open('downton.abbey.s01e01.srt','r')
# content = f.read()
# group = [group.split("\n",2) for group in content.split("\n\n")]
# print(group)

[['1', '00:01:10,780 --> 00:01:13,060', 'Oh, my God!'], ['2', '00:01:16,580 --> 00:01:18,580', "That's impossible."], ['3', '00:01:19,280 --> 00:01:20,580', "I'll take it up there now."], ['4', '00:01:21,080 --> 00:01:23,690', "Don't be stupid. None of\nthem will be up for hours,"], ['5', '00:01:23,700 --> 00:01:25,340', 'and what difference will it make?'], ['6', '00:01:25,350 --> 00:01:27,360', "Jimmy'll do it when he comes in."], ['7', '00:01:41,470 --> 00:01:44,060', 'April 1912'], ['8', '00:01:45,420 --> 00:01:47,380', "Six o'clock!"], ['9', '00:01:49,220 --> 00:01:52,240', 'Thank you, Daisy... Anna?'], ['10', '00:01:56,120 --> 00:01:57,610', 'Just for once in my life,'], ['11', '00:01:57,620 --> 00:02:00,480', "I'd like to sleep\nuntil I woke up natural."], ['12', '00:02:01,380 --> 00:02:04,480', '- Is your fire still in?\n- Yes, Mrs. Patmore.'], ['13', '00:02:04,490 --> 00:02:06,630', 'Ooh, my, will wonders never cease?'], ['14', '00:02:06,660 --> 00:02:09,410', "- Have you laid

In [ ]:
# pairs1 = dialogue_pairs(group)

NameError: ignored

In [ ]:
def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.replace("\n"," ") 
    
    return text


In [ ]:
for p in pairs1:
  p[0] = clean_text(p[0])
  p[1] = clean_text(p[1])

In [ ]:
print(pairs1[:10])

[['font color3399ccsubtitles by fontfont colorffffffmemoryonsmellsfont font color3399cchttpuksubtitlesrufont', 'should not the hot toast come to me first daisy sorry mr carson'], ['should not the hot toast come to me first daisy sorry mr carson', 'why is alfred getting special treatment i am sure he is not'], ['why is alfred getting special treatment i am sure he is not', 'why is he getting special treatment'], ['why is he getting special treatment', 'why do you think because he is staying here with us'], ['why do you think because he is staying here with us', 'oh this is nice'], ['oh this is nice', 'mr napier and mr blake are coming up in a few days'], ['mr napier and mr blake are coming up in a few days', 'he asks if i am serious about their staying here'], ['he asks if i am serious about their staying here', 'and are you'], ['and are you', 'certainly they could be useful'], ['certainly they could be useful', 'they are writing a report on why estates like this are going wrong']]


In [ ]:
THRESHOLD = 3
MAX_SENTENCE_LENGTH = 10
MAX_LENGTH = 10

In [ ]:
pairs1 = [p for p in pairs1 if len(p[0].split()) <= MAX_SENTENCE_LENGTH and len(p[1].split()) <= MAX_SENTENCE_LENGTH]

In [ ]:
pairs1

[['he asks if i am serious about their staying here', 'and are you'],
 ['and are you', 'certainly they could be useful'],
 ['but i would not mind having it confirmed', 'you seem brighter lately'],
 ['you seem brighter lately', 'have things sorted themselves out'],
 ['have things sorted themselves out', 'not quite but'],
 ['not quite but', 'it is better'],
 ['it is better', 'yes'],
 ['yes', 'and you have moved back into the cottage'],
 ['and you have moved back into the cottage', 'i have my lady'],
 ['but i am glad if it is resolved', 'nothing for me'],
 ['nothing for me', 'afraid not'],
 ['afraid not', 'this is a fine kettle of fish i must say'],
 ['this is a fine kettle of fish i must say',
  'your uncle harold is in a proper fix'],
 ["intensive farming's a big step", 'half past eight i must skedaddle'],
 ['half past eight i must skedaddle', 'penny for your thoughts'],
 ['penny for your thoughts', 'you would pay twice that not to know them'],
 ['you would pay twice that not to know th

In [ ]:
queries = [p[0] for p in pairs1]
responses = [p[1] for p in pairs1]
print(queries)
query_words = []
# query_words = [q.split() for q in queries]
for q in queries:
  query_words.extend(q.split())
print(query_words)
query_vocab = Counter(query_words)

response_words = []
# query_words = [q.split() for q in queries]
for r in responses:
  response_words.extend(r.split())
print(response_words)
response_vocab = Counter(response_words)

print(query_vocab)
print(response_vocab)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# queries = [p for p in queries if len(p) < MAX_SENTENCE_LENGTH]
# responses = [p for p in responses if len(p) < MAX_SENTENCE_LENGTH]


In [ ]:
query_word_to_int = {}

word_int_label = 0
for word in query_vocab:
  if query_vocab[word] < THRESHOLD:
    continue

  else:
    query_word_to_int[word] = word_int_label
    word_int_label += 1

response_word_to_int = {}

word_int_label = 0
for word in response_vocab:
  if response_vocab[word] < THRESHOLD:
    continue

  else:
    response_word_to_int[word] = word_int_label
    word_int_label += 1
  

print(query_word_to_int)
print(response_word_to_int)
# EOS_token = 2  # End-of-sentence token

# class Voc:

{'he': 0, 'asks': 1, 'if': 2, 'i': 3, 'am': 4, 'serious': 5, 'about': 6, 'their': 7, 'staying': 8, 'here': 9, 'and': 10, 'are': 11, 'you': 12, 'but': 13, 'would': 14, 'not': 15, 'mind': 16, 'having': 17, 'it': 18, 'seem': 19, 'lately': 20, 'have': 21, 'things': 22, 'themselves': 23, 'out': 24, 'quite': 25, 'is': 26, 'better': 27, 'yes': 28, 'moved': 29, 'back': 30, 'into': 31, 'the': 32, 'cottage': 33, 'glad': 34, 'resolved': 35, 'nothing': 36, 'for': 37, 'me': 38, 'afraid': 39, 'this': 40, 'a': 41, 'fine': 42, 'kettle': 43, 'of': 44, 'fish': 45, 'must': 46, 'say': 47, 'big': 48, 'step': 49, 'half': 50, 'past': 51, 'eight': 52, 'penny': 53, 'your': 54, 'thoughts': 55, 'pay': 56, 'twice': 57, 'that': 58, 'to': 59, 'know': 60, 'them': 61, 'there': 62, 'was': 63, 'anything': 64, 'could': 65, 'do': 66, 'fault': 67, 'mine': 68, 'husband': 69, 'making': 70, 'sauce': 71, 'souffle': 72, 'mrs': 73, 'hughes': 74, 'one': 75, 'secret': 76, 'mean': 77, 'little': 78, 'touched': 79, 'maids': 80, 'cou

In [ ]:
spec_tokens = ['<PAD>','<SOS>','<EOS>','<UNK>']

for token in spec_tokens:
  query_word_to_int[token] = len(query_word_to_int)+1

for token in spec_tokens:
  response_word_to_int[token] = len(response_word_to_int)+1

In [ ]:
# Inverse dictionary
query_int_to_word = {v:k for k,v in query_word_to_int.items()}
responses_int_to_word = {v:k for k,v in response_word_to_int.items()}
query_int_to_word.update(responses_int_to_word)

In [ ]:
# pairs = list(zip(queries,responses))
pairs = []
for p in list(zip(queries,responses)):
  keep_query = True
  keep_response = True
  for x in p[0].split():
    if x not in query_word_to_int:
      keep_query = False
      break

  if not keep_query: continue
  for x in p[1].split():
    if x not in response_word_to_int:
      keep_response = False
      break

  if keep_query and keep_response:
    pairs.append(p)

print(pairs)


  

# for q in queries:
#   lst = [x for x in q if query_vocab[x] >= THRESHOLD])
#   if not lst: 
#     queries.remove(q)
#   else:
#     q = q + ' <EOS>'

# for r in responses:
#   r = ''.join([x for x in r if response_vocab[x] >= THRESHOLD])
#   if not r: 
#     responses.remove(r)
#   else:
#     r = r + ' <EOS>'

[('he asks if i am serious about their staying here', 'and are you'), ('and are you', 'certainly they could be useful'), ('not quite but', 'it is better'), ('it is better', 'yes'), ('yes', 'and you have moved back into the cottage'), ('and you have moved back into the cottage', 'i have my lady'), ('but i am glad if it is resolved', 'nothing for me'), ('nothing for me', 'afraid not'), ('afraid not', 'this is a fine kettle of fish i must say'), ('this is a fine kettle of fish i must say', 'your uncle harold is in a proper fix'), ('penny for your thoughts', 'you would pay twice that not to know them'), ('you would pay twice that not to know them', 'if there was anything i could do i know'), ('if there was anything i could do i know', 'it is not your fault it is mine'), ('mrs hughes is the one for a secret', 'yes that is true'), ('i know it but i have not touched it', 'not to wash it or mend it or something'), ('that is the risk with little things', 'they fit into a pocket easily'), ('lord

In [ ]:
# pairs = list(zip(queries,responses))
# print(pairs)

[('oh my god', 'that is impossible'), ('that is impossible', 'i will take it up there now'), ('and what difference will it make', 'jimmy will do it when he comes in'), ('april 1912', "six o'clock"), ("six o'clock", 'thank you daisy anna'), ('thank you daisy anna', 'just for once in my life'), ('just for once in my life', 'i would like to sleep until i woke up natural'), ('i would like to sleep until i woke up natural', ' is your fire still in  yes mrs patmore'), (' is your fire still in  yes mrs patmore', 'ooh my will wonders never cease'), ('ooh my will wonders never cease', " have you laid the servants' hall breakfast  yes mrs patmore"), (" have you laid the servants' hall breakfast  yes mrs patmore", ' and finished blacking that stove  yes mrs patmore'), (' and finished blacking that stove  yes mrs patmore', ' what about the bedroom fires  all lit mrs patmore'), (' what about the bedroom fires  all lit mrs patmore', 'right well take the things'), ('right well take the things', 'and 

In [ ]:
query_ints = []
for q in queries:
  temp = []
  for word in q:
    try:
      temp.append(query_word_to_int[word])
    except:
      temp.append(query_word_to_int['<UNK>'])
    query_ints.append(temp)

response_ints = []
for q in responses:
  temp = []
  for word in q:
    try:
      temp.append(query_word_to_int[word])
    except:
      temp.append(query_word_to_int['<UNK>'])
    response_ints.append(temp)

In [ ]:
pairs

[('he asks if i am serious about their staying here', 'and are you'),
 ('and are you', 'certainly they could be useful'),
 ('not quite but', 'it is better'),
 ('it is better', 'yes'),
 ('yes', 'and you have moved back into the cottage'),
 ('and you have moved back into the cottage', 'i have my lady'),
 ('but i am glad if it is resolved', 'nothing for me'),
 ('nothing for me', 'afraid not'),
 ('afraid not', 'this is a fine kettle of fish i must say'),
 ('this is a fine kettle of fish i must say',
  'your uncle harold is in a proper fix'),
 ('penny for your thoughts', 'you would pay twice that not to know them'),
 ('you would pay twice that not to know them',
  'if there was anything i could do i know'),
 ('if there was anything i could do i know',
  'it is not your fault it is mine'),
 ('mrs hughes is the one for a secret', 'yes that is true'),
 ('i know it but i have not touched it',
  'not to wash it or mend it or something'),
 ('that is the risk with little things', 'they fit into a 

In [ ]:
random.choice(pairs)

IndexError: ignored

In [ ]:
def indexesFromSentence(voc, sentence):
    result = []
    for word in sentence.split():
        try:
          result.append(voc[word])
        except:
          # result.append(voc['<UNK>'])
          continue

    return result
    # return [voc[word] for word in sentence.split()]



def zeroPadding(l, voc,fillvalue='<PAD>'):
    return list(itertools.zip_longest(*l, fillvalue=voc[fillvalue]))

def binaryMatrix(l, value='<PAD>'):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == '<PAD>':
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch,voc)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch,voc)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
query_word_to_int.update(response_word_to_int)
batches = batch2TrainData(query_word_to_int, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[ 261,   26,  128,    1,   17],
        [ 607,  527,   18,    2,   18],
        [ 397,   26,   24,  260,   72],
        [  26,  366, 2344,   22,   60],
        [ 265,  105,  146,   56, 2893],
        [ 129,    2,   26,  317, 2893],
        [ 462,  111,  235,  177, 2893],
        [ 216,  110, 2893, 2893, 2893],
        [1795,   56, 2893, 2893, 2893],
        [1275,  763, 2893, 2893, 2893]])
lengths: tensor([10, 10,  7,  7,  4])
target_variable: tensor([[   0,  432,   26,   71,  652],
        [  17,  280,   63,   26,  459],
        [ 280,    2,   14,  103,   60],
        [ 909,  290,   57,  641,  111],
        [   2,   26, 2893,   87,   14],
        [ 234,  196, 2893,   27,  469],
        [   0,   56, 2893,  559,  462],
        [2545,  136, 2893, 2893, 2893],
        [2893,  968, 2893, 2893, 2893]])
mask: tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
 

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths,enforce_sorted=False)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [ ]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [ ]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [ ]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[query_word_to_int['<SOS>'] for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [ ]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * query_word_to_int['<SOS>']
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [query_int_to_word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = clean_text(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(len(query_word_to_int.keys()), hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, len(query_word_to_int.keys()), decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [ ]:
save_dir = os.path.join("data", "save")
corpus_name = "downton abbey"

In [ ]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
    
# Run training iterations
print("Starting Training!")
trainIters(model_name, query_word_to_int, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 8.1033
Iteration: 2; Percent complete: 0.1%; Average loss: 7.8186
Iteration: 3; Percent complete: 0.1%; Average loss: 7.4863
Iteration: 4; Percent complete: 0.1%; Average loss: 7.0262
Iteration: 5; Percent complete: 0.1%; Average loss: 6.4066
Iteration: 6; Percent complete: 0.1%; Average loss: 5.6949
Iteration: 7; Percent complete: 0.2%; Average loss: 5.3530
Iteration: 8; Percent complete: 0.2%; Average loss: 5.4781
Iteration: 9; Percent complete: 0.2%; Average loss: 5.4393
Iteration: 10; Percent complete: 0.2%; Average loss: 5.7749
Iteration: 11; Percent complete: 0.3%; Average loss: 4.7303
Iteration: 12; Percent complete: 0.3%; Average loss: 4.7451
Iteration: 13; Percent complete: 0.3%; Average loss: 4.8237
Iteration: 14; Percent complete: 0.4%; Average loss: 4.6835
Iteration: 15; Percent complete: 0.4%; Average loss: 4.5310
Iteration: 16; Percent complete: 0.4%

In [ ]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
voc = query_word_to_int
evaluateInput(encoder, decoder, searcher, voc)

> hello
Bot: hello miss denker that is your mission today <PAD> <PAD>
> what
Bot: lady mary crawley not at all <PAD> <PAD> <PAD> <PAD>
> oh no
Bot: we are down to the bare bones as it is
> how are you
Bot: really <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
> help
Bot: i am afraid we must be going <PAD> <PAD> <PAD>
> please
Bot: oh no <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
> how are we feeling today
Bot: pretty <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
> good
Bot: how was she i do not know <PAD> <PAD> <PAD>
> i do not know
Bot: maybe sometimes <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
> sometimes what
Bot: is that all <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
> no not yet
Bot: i am not asking for properly <PAD> <PAD> <PAD> <PAD>
> then ask
Bot: well it is a long time you know <PAD> <PAD>
> i know
Bot: but i do not like it <PAD> <PAD> <PAD> <PAD>
> neither do i
Bot: you must have second money <PAD> <PAD> <PAD> <PAD> <PAD>
> no I don't
Bot: oh my god e